In [3]:
import pandas as pd
from requests import request
import json
from datetime import datetime
today_timestamp = datetime.now().strftime("%Y%m%d")

In [3]:
# Run this to download newest data

# Read credentials file
with open("credentials.json", mode="r") as creds_file:
    credentials = json.load(creds_file)
url = credentials["url"]

headers = credentials["headers"]

# Get response
response = request("GET", url, headers=headers)

# Check if response was successful
if response.status_code == 200:
    # Get the data from the API response
    data = response.json()
    
    # Write the data to a JSON file
    file_path = f"downloads/raw_data_{today_timestamp}.json"
    with open(file_path, "w") as file:
        json.dump(data, file)
        
    # Print the dataframe
    # df.to_csv("data.csv", index=False)
else:
    # Handle the error
    print('Failed to retrieve data from API. Error code:', response.status_code)

In [12]:
# Update data file with the newest

# Read the data file
with open("data.json", mode="r") as saved_data_file:
    saved_data = json.load(saved_data_file)

# Save current file in archive
archive_path = f"archive/data_{today_timestamp}.json"
with open(archive_path, mode="w") as archive_file:
    json.dump(saved_data, archive_file)

with open("data_20230429.json", mode="r") as raw_file:
    temp_data = json.load(raw_file)[0]

# Clean temp_data
cleansed_temp_data = {}
for matchday, game_list in temp_data.items():
    cleansed_temp_data[matchday.strip()] = game_list

updated_data =  saved_data | cleansed_temp_data 

with open("data.json", mode="w") as data_file:
    json.dump(updated_data, data_file)

In [2]:
with open("data_20230429.json", mode="r") as data_file:
    saved_data = json.load(data_file)[0]

cleansed_data = {}
for matchday, game_list in saved_data.items():
    cleansed_data[matchday.strip()] = game_list
    # cleansed_data[matchday.strip()] = game[0]
# print(cleansed_data["Matchday 32"])

for matchday, game_list in cleansed_data.items():
    print(matchday)
    for game in game_list:
        # print(game)
        print(f"{game['homeTeam']} {game['homeTeamScore']}-{game['awayTeamScore']} {game['awayTeam']}")
    print("<><><><><><>")
with open("cd.json", "w") as file:
    json.dump(cleansed_data, file)    

# for matchday, game in cleansed_data.items():
#     print(f"{game['homeTeam']} {game['homeTeamScore']}-{game['awayTeamScore']} {game['awayTeam']}\n")

Matchday 32
Bologna - Juventus
Fiorentina - Sampdoria
Sassuolo - Empoli
Napoli - Salernitana
Cremonese - Hellas Verona
Inter - Lazio
Torino - Atalanta
Roma - Milan
Spezia 0-2 Monza
Lecce 1-0 Udinese
<><><><><><>
Matchday 31
Atalanta 3-1 Roma
Juventus 0-1 Napoli
Milan 2-0 Lecce
Udinese 3-0 Cremonese
Monza 3-2 Fiorentina
Empoli 0-3 Inter
Sampdoria 1-1 Spezia
Lazio 0-1 Torino
Salernitana 3-0 Sassuolo
Hellas Verona 2-1 Bologna
<><><><><><>
Matchday 30
Fiorentina 1-1 Atalanta
Roma 3-0 Udinese
Sassuolo 1-0 Juventus
Torino 1-1 Salernitana
Lecce 1-1 Sampdoria
Inter 0-1 Monza
Napoli 0-0 Hellas Verona
Bologna 1-1 Milan
Spezia 0-3 Lazio
Cremonese 1-0 Empoli
<><><><><><>
Matchday 29
Lazio 2-1 Juventus
Torino 0-1 Roma
Hellas Verona 2-1 Sassuolo
Sampdoria 2-3 Cremonese
Atalanta 0-2 Bologna
Fiorentina 1-1 Spezia
Udinese 2-2 Monza
Milan 0-0 Empoli
Lecce 1-2 Napoli
Salernitana 1-1 Inter
<><><><><><>


In [4]:
tester = saved_data
# print(tester)
print(type(tester))

for matchday in tester:
    print(f"\n{matchday}\n".strip())
    for game in tester[matchday]:
        # print the game result
        print(f"{game['homeTeam']} {game['homeTeamScore']}-{game['awayTeamScore']} {game['awayTeam']}\n")
        ht = tester[matchday][0]

<class 'dict'>
Matchday 32
Bologna - Juventus

Fiorentina - Sampdoria

Sassuolo - Empoli

Napoli - Salernitana

Cremonese - Hellas Verona

Inter - Lazio

Torino - Atalanta

Roma - Milan

Spezia 0-2 Monza

Lecce 1-0 Udinese

Matchday 31
Atalanta 3-1 Roma

Juventus 0-1 Napoli

Milan 2-0 Lecce

Udinese 3-0 Cremonese

Monza 3-2 Fiorentina

Empoli 0-3 Inter

Sampdoria 1-1 Spezia

Lazio 0-1 Torino

Salernitana 3-0 Sassuolo

Hellas Verona 2-1 Bologna

Matchday 30
Fiorentina 1-1 Atalanta

Roma 3-0 Udinese

Sassuolo 1-0 Juventus

Torino 1-1 Salernitana

Lecce 1-1 Sampdoria

Inter 0-1 Monza

Napoli 0-0 Hellas Verona

Bologna 1-1 Milan

Spezia 0-3 Lazio

Cremonese 1-0 Empoli

Matchday 29
Lazio 2-1 Juventus

Torino 0-1 Roma

Hellas Verona 2-1 Sassuolo

Sampdoria 2-3 Cremonese

Atalanta 0-2 Bologna

Fiorentina 1-1 Spezia

Udinese 2-2 Monza

Milan 0-0 Empoli

Lecce 1-2 Napoli

Salernitana 1-1 Inter

